In [1]:
pip install -q transformers accelerate flash_attn 


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import LlavaProcessor, LlavaForConditionalGeneration
import torch
model_id = "llava-hf/llava-interleave-qwen-0.5b-hf"

processor = LlavaProcessor.from_pretrained(model_id)

model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16)
model.to("cuda") # can also be xpu, mps, npu etc. depending on your hardware accelerator

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


LlavaForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(729, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-25): 26 x SiglipEncoderLayer(
            (self_attn): SiglipSdpaAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=115

In [3]:
model

LlavaForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(729, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-25): 26 x SiglipEncoderLayer(
            (self_attn): SiglipSdpaAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=115

In [4]:
import uuid
import requests
import cv2
from PIL import Image

def replace_video_with_images(text, frames):
  return text.replace("<video>", "<image>" * frames)

def sample_frames(url, num_frames):

    response = requests.get(url)
    path_id = str(uuid.uuid4())

    path = f"./{path_id}.mp4" 

    with open(path, "wb") as f:
      f.write(response.content)

    video = cv2.VideoCapture(path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // num_frames
    frames = []
    for i in range(total_frames):
        ret, frame = video.read()
        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if not ret:
            continue
        if i % interval == 0:
            frames.append(pil_img)
    video.release()
    return frames[:num_frames]

In [5]:
video_1 = "https://huggingface.co/spaces/merve/llava-interleave/resolve/main/cats_1.mp4"
video_2 = "https://huggingface.co/spaces/merve/llava-interleave/resolve/main/cats_2.mp4"

video_1 = sample_frames(video_1, 6)
video_2 = sample_frames(video_2, 6)

videos = video_1 + video_2

videos

# [<PIL.Image.Image image mode=RGB size=1920x1080>,
# <PIL.Image.Image image mode=RGB size=1920x1080>,
# <PIL.Image.Image image mode=RGB size=1920x1080>, ...]

[<PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>]

In [6]:
user_prompt = "Are these two cats in these two videos doing the same thing?"
toks = "<image>" * 12
prompt = "<|im_start|>user"+ toks + f"\n{user_prompt}<|im_end|><|im_start|>assistant"
inputs = processor(text=prompt, images=videos, return_tensors="pt").to(model.device, model.dtype)

In [7]:
inputs

{'input_ids': tensor([[151644,    872, 151646,  ..., 151645, 151644,  77091]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[[[-0.4902, -0.4902, -0.4902,  ..., -0.6782, -0.6782, -0.6782],
          [-0.4902, -0.4902, -0.4902,  ..., -0.6782, -0.6782, -0.6782],
          [-0.4902, -0.4902, -0.4902,  ..., -0.6782, -0.6782, -0.6782],
          ...,
          [ 0.4824,  0.4980,  0.5059,  ...,  0.6392,  0.6392,  0.6470],
          [ 0.4746,  0.4902,  0.4980,  ...,  0.6392,  0.6392,  0.6470],
          [ 0.4746,  0.4902,  0.4980,  ...,  0.6392,  0.6392,  0.6470]],

         [[-0.6626, -0.6626, -0.6626,  ..., -0.7881, -0.7881, -0.7881],
          [-0.6626, -0.6626, -0.6626,  ..., -0.7881, -0.7881, -0.7881],
          [-0.6626, -0.6626, -0.6626,  ..., -0.7881, -0.7881, -0.7881],
          ...,
          [ 0.3098,  0.3254,  0.3333,  ...,  0.5605,  0.5605,  0.5688],
          [ 0.3020,  0.3176,  0.3254,  ...,  0.5605,  0.5

In [8]:
output = model.generate(**inputs, max_new_tokens=100, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True)[len(user_prompt)+10:])

# The first cat is shown in a relaxed state, with its eyes closed and a content expression, while the second cat is shown in a more active state, with its mouth open wide, possibly in a yawn or a vocalization.



Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


 No, the two cats in the video are not engaging in the same activity. One cat is lying down while the other is standing.
